Preparing "worst" and "best" subsets per category in dataset bases on hash distance metrics

In [1]:
import numpy as np
#see create_training_paths for creation of the precalculated dcts; 
#due to copyright these may present an issue when uploaded/clash with the open license -> you have to repeat the script yourself
cifar10_hashes = np.load('cifar10_hashes.npz')

In [2]:
hashes, paths = cifar10_hashes['hashes'], cifar10_hashes['paths']

In [3]:
from cifar10_trainer import get_cat
idx = {}
for i,p in enumerate(paths[10000:]):
    idx.setdefault(get_cat(p),[]).append(i+10000)
idx = {k:[min(v),max(v)] for k,v in idx.items()}

In [4]:
idx

{'airplane': [10000, 14999],
 'automobile': [15000, 19999],
 'bird': [20000, 24999],
 'cat': [25000, 29999],
 'deer': [30000, 34999],
 'dog': [35000, 39999],
 'frog': [40000, 44999],
 'horse': [45000, 49999],
 'ship': [50000, 54999],
 'truck': [55000, 59999]}